In [1]:
from IPython import get_ipython
from typing_extensions import Annotated

import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-1106"],
    },
)
print(config_list)

[{'model': 'gpt-4-1106', 'api_key': '7b81e8a5d22142f1a69845f53a8d16c8', 'base_url': 'https://canandaeast.openai.azure.com/', 'api_type': 'azure', 'api_version': '2023-12-01-preview'}]


### Database Agent

In [2]:

import os
import pandas as pd
import sys
from io import StringIO
import autogen
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Text, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import random
from datetime import datetime

engine = create_engine('sqlite:///data/sqlite.db') 

def execute_python_code(python_code, goal):
    def execute_sql_query(sql_query, limit=100):  
        result = pd.read_sql_query(sql_query, engine)
        result = result.infer_objects()
        for col in result.columns:  
            if 'date' in col.lower():  
                result[col] = pd.to_datetime(result[col], errors="ignore")  

        if limit is not None:  
            result = result.head(limit)  # limit to save memory  
        return result

    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    new_input=""
    try:
        exec(python_code, locals())
        sys.stdout = old_stdout
        std_out = str(mystdout.getvalue())
        if len(std_out)>0:
            new_input +="\n"+ std_out 
    except Exception as e:
        new_input +="\Encounter following error, please fix the bug and give updated code\n"+str(e)+"\n"
    if len(new_input)==0:
        new_input = "No output from your code, please check if you have any print() statement in your code\n"
    return new_input

database_schema ="""

For a Database Server log table, the focus is often on capturing query execution details, transaction information, and potential errors or performance issues. Here's schema for a database server log table:

CREATE TABLE DatabaseLogs (
    DBLogID INT AUTO_INCREMENT PRIMARY KEY,
    Timestamp DATETIME NOT NULL,
    DatabaseName VARCHAR(255),
    UserName VARCHAR(255),
    ClientHost VARCHAR(255),
    QueryText TEXT,
    QueryType VARCHAR(50),
    ExecutionTime INT,
    RowsAffected INT,
    TransactionID VARCHAR(255),
    ErrorCode INT,
    ErrorDescription TEXT,
    QueryPlan TEXT,
    LockWaitTime INT,
    Deadlock BOOLEAN,
    CPUUsage INT,
    MemoryUsage INT,
    DiskIO INT,
    NetworkIO INT,
    AdditionalInfo TEXT
);


Column Descriptions:

DBLogID: A unique identifier for each log entry.
Timestamp: The date and time when the query was executed.
DatabaseName: The name of the database on which the query was executed.
UserName: The username of the individual who executed the query.
ClientHost: The hostname or IP address of the client that initiated the query.
QueryText: The actual SQL query text that was executed.
QueryType: The type of operation (e.g., SELECT, INSERT, UPDATE, DELETE).
ExecutionTime: The time taken to execute the query in milliseconds.
RowsAffected: The number of rows affected by the query.
TransactionID: A unique identifier for the transaction within which this query was executed.
ErrorCode: The error code returned if the query resulted in an error.
ErrorDescription: A description of the error if one occurred.
QueryPlan: Details about the execution plan for the query, which can be useful for performance analysis.
LockWaitTime: The time spent waiting for locks, if applicable.
Deadlock: A boolean flag indicating whether the query was involved in a deadlock situation.
CPUUsage: The amount of CPU time consumed by the query.
MemoryUsage: The amount of memory used during the query's execution.
DiskIO: The amount of disk I/O generated by the query.
NetworkIO: The amount of network I/O generated, if the query involved distributed or remote resources.
AdditionalInfo: Any other relevant information that could be helpful for troubleshooting or performance tuning.

."""

database_function_spec = [

        {
        "type":"function",
        "function":{

        "name": "execute_python_code",
        "description": "execute a python code for data analysis",
        "parameters": {
            "type": "object",
            "properties": {
                "python_code": {
                    "type": "string",
                    "description": f"python code snippet that can be executed. You are provided with a utility function execute_sql_query(sql_query) to execute sql query and return the result as a pandas dataframe so use it to retrieve data you need and don't try to connect to the database. In the end, remember to use print() function to observe any output. Here is the schema of the database logs table so that you can write the query correctly:\n{database_schema}"
                },
                "goal": {
                    "type": "string",
                    "description": "description of what you hope to achieve with this python code snippset"
                }

            },
            "required": ["python_code", "goal"],
        },

    }
    }
    
]
database_available_functions = {
            "execute_python_code": execute_python_code,
        } 

database_expert_system_message = f"""
You are database system expert. You help retrieve system metrics by writing python code to query and analyze data from log table.

"""
database_expert = autogen.AssistantAgent(
    name="database_expert",
    llm_config={"config_list": config_list, "tools":database_function_spec},
    # the default system message of the AssistantAgent is overwritten here
    system_message=database_expert_system_message,
    
)
def is_termination_msg(x):
    if  x.get("tool_calls") is None and x.get("role") =="user":
            print("return true")
            return True
    else:
        print("return false")
        return False


database_user = autogen.UserProxyAgent(
    name="database_user",
    is_termination_msg=is_termination_msg,
    human_input_mode="NEVER",
)
database_user.register_function(function_map= database_available_functions)




In [3]:
database_user.initiate_chat(database_expert, message="show me top 10 slowest running queries")

database_user (to database_expert):

show me top 10 slowest running queries

--------------------------------------------------------------------------------
database_expert (to database_user):

***** Suggested tool Call (call_FpgdMgGqatUXQbzAjrgfF04j): execute_python_code *****
Arguments: 
{
  "python_code": "import pandas as pd\n\n# Define the SQL query to retrieve the top 10 slowest running queries.\nsql_query = \"\"\"\nSELECT *\nFROM DatabaseLogs\nORDER BY ExecutionTime DESC\nLIMIT 10\n\"\"\"\n\n# Execute the SQL query and retrieve the result in a pandas dataframe.\ndf_slowest_queries = execute_sql_query(sql_query)\n\n# Display the result.\nprint(df_slowest_queries)",
  "goal": "Retrieve and display the top 10 slowest running queries from the database logs."
}
************************************************************************************

--------------------------------------------------------------------------------
return false

>>>>>>>> EXECUTING FUNCTION execute_python_c

### Webserver Agent

In [4]:

webserver_schema ="""

For web server logs, it is common to capture HTTP request and response data, along with server performance metrics. Here's an example schema for a web server log table:

CREATE TABLE WebServerLogs (
    WebLogID INT AUTO_INCREMENT PRIMARY KEY,
    Timestamp DATETIME NOT NULL,
    Hostname VARCHAR(255),
    ServerIP VARCHAR(15),
    ClientIP VARCHAR(15),
    UserAgent VARCHAR(512),
    RequestMethod VARCHAR(10),
    RequestURL TEXT,
    RequestProtocol VARCHAR(10),
    StatusCode INT,
    ResponseSize BIGINT,
    ReferrerURL TEXT,
    SessionID VARCHAR(255),
    UserID INT,
    ResponseTime INT,
    SSLProtocol VARCHAR(50),
    TLSCipher VARCHAR(100),
    ErrorLog TEXT,
    AdditionalInfo TEXT
);

Column Descriptions:

WebLogID: A unique identifier for each log entry.
Timestamp: The date and time when the request was processed.
Hostname: The hostname of the server that processed the request.
ServerIP: The IP address of the server.
ClientIP: The IP address of the client making the request.
UserAgent: The user agent string of the client's browser or tool making the request.
RequestMethod: The HTTP method used (e.g., GET, POST, PUT, DELETE).
RequestURL: The URL that was requested.
RequestProtocol: The protocol used for the request (e.g., HTTP/1.1, HTTP/2, HTTPS).
StatusCode: The HTTP status code returned (e.g., 200, 404, 500).
ResponseSize: The size of the response in bytes.
ReferrerURL: The referrer URL if provided by the client.
SessionID: A unique identifier for the user session.
UserID: A system identifier for the user making the request, if authenticated.
ResponseTime: The time taken to serve the request in milliseconds.
SSLProtocol: The SSL protocol used for secure requests (e.g., TLS 1.2, TLS 1.3).
TLSCipher: The TLS cipher suite used for the request, if applicable.
ErrorLog: Any error messages or stack traces if the request resulted in an error.
AdditionalInfo: Any other relevant information that might assist in diagnosing issues or analyzing traffic patterns.

This schema captures a range of data that can be used for troubleshooting, security analysis, performance monitoring, and understanding user behavior on the web server. Depending on the level of detail required and the specific use cases, more fields could be added, such as those capturing cookie data, full request and response headers, or more detailed timing information for various stages of request handling.
"""

webserver_function_spec = [

        {
        "type":"function",
        "function":{

        "name": "execute_python_code",
        "description": "execute a python code for data analysis",
        "parameters": {
            "type": "object",
            "properties": {
                "python_code": {
                    "type": "string",
                    "description": f"python code snippet that can be executed. You are provided with a utility function execute_sql_query(sql_query) to execute sql query and return the result as a pandas dataframe so use it to retrieve data you need and don't try to connect to the database. In the end, remember to use print() function to observe any output. Here is the schema of the webserver logs table so that you can write the query correctly:\n{webserver_schema}"
                },
                "goal": {
                    "type": "string",
                    "description": "description of what you hope to achieve with this python code snippset"
                }

            },
            "required": ["python_code", "goal"],
        },

    }
    }
    
]
webserver_available_functions = {
            "execute_python_code": execute_python_code,
        } 

webserver_expert_system_message = f"""
You are webserver system expert. You help retrieve web system metrics by writing python code to query and analyze data from log table.

"""
webserver_expert = autogen.AssistantAgent(
    name="webserver_expert",
    llm_config={"config_list": config_list, "tools":webserver_function_spec},
    # the default system message of the AssistantAgent is overwritten here
    system_message=webserver_expert_system_message,
    
)
def is_termination_msg(x):
    if  x.get("tool_calls") is None and x.get("role") =="user":
            print("return true")
            return True
    else:
        print("return false")
        return False


webserver_user = autogen.UserProxyAgent(
    name="webserver_user",
    is_termination_msg=is_termination_msg,
    human_input_mode="NEVER",
)
webserver_user.register_function(function_map= webserver_available_functions)


# ask_websever_expert("I want to know the number of requests per month")

### Main Agent

In [5]:
main_agent_system_message = """
You are a customer support specialist that help customers understand the cause of their techical issues.
First, you need to look up the guideline for the issue in the knowledge base.
Then follow the guideline to help the customer.
You have access to multiple assistants who can help you query specific information.
Once you gather sufficient information, give customer a better understanding of the issue or a solution.

"""
main_agent = autogen.AssistantAgent(
    name="planner",
    llm_config={"config_list": config_list},
    # the default system message of the AssistantAgent is overwritten here
    system_message=main_agent_system_message,
)



main_agent_proxy = autogen.UserProxyAgent(
    is_termination_msg=is_termination_msg,
    name="planner_user",
    human_input_mode="NEVER",
)

@main_agent_proxy.register_for_execution()
@main_agent.register_for_llm(name="look_up_guideline", description="Look up investigation guideline for the issue")
def look_up_guideline(customer_issue: Annotated[str, "summary of customer issue"])-> str:
    guideline = """
### Scenario 2: Slow Performance in Web Application

**Customer Trouble:**
Users are reporting that the web application's performance has significantly degraded, with page load times being much longer than usual.
**Systems Involved:**
1. Web Servers
2. Database Servers
**Specific Guidance for Analysis:**
- **Web Servers:** Check for any issue with web server such as response time, error etc...
- **Database Servers:** Analyze query performance and look for slow-running queries or table scans that could be causing bottlenecks. Evaluate the indexing strategy and optimize queries as needed.
    """
    return guideline
@main_agent_proxy.register_for_execution()
@main_agent.register_for_llm(name="ask_database_expert", description="Ask database for any performance metrics of the database server")
def ask_database_expert(question: Annotated[str, "Specific question to understand any metrics and status of database system"])-> str:

    print("question for datbase is ", question)
    database_user.initiate_chat(database_expert, message=question)
    # return the last message received from the planner
    answer_from_database = database_user.last_message()["content"]
    print("answer from datagbase is ", answer_from_database)
    return answer_from_database

@main_agent_proxy.register_for_execution()
@main_agent.register_for_llm(name="ask_websever_expert", description="Ask websever expert for any performance metrics of the web server")
def ask_websever_expert(question: Annotated[str, "Question to ask webserver expert to provide"])-> str:
    print("question for webserver is ", question)
    webserver_user.initiate_chat(webserver_expert, message=question)
    # return the last message received from the planner
    answer_from_webserver = webserver_user.last_message()["content"]
    print("answer from webserver is ", answer_from_webserver)
    return answer_from_webserver

main_agent_proxy.initiate_chat(main_agent, message="Application has been running slow, averge response time is more than 100 ms, can you help me understand what is going on?")

planner_user (to planner):

Application has been running slow, averge response time is more than 100 ms, can you help me understand what is going on?

--------------------------------------------------------------------------------
planner (to planner_user):

***** Suggested tool Call (call_50DxEp08xtFpNvQX6yIEMmAO): look_up_guideline *****
Arguments: 
{"customer_issue":"Application slow response time"}
**********************************************************************************

--------------------------------------------------------------------------------
return false

>>>>>>>> EXECUTING FUNCTION look_up_guideline...
planner_user (to planner):

planner_user (to planner):

***** Response from calling tool "look_up_guideline" *****

### Scenario 2: Slow Performance in Web Application

**Customer Trouble:**
Users are reporting that the web application's performance has significantly degraded, with page load times being much longer than usual.
**Systems Involved:**
1. Web Servers

In [6]:
print(main_agent_proxy.last_message()["content"])

The investigation into the application's performance issue has revealed the following information:

1. **Web Server's Performance**:
   - The current average response time on the web server is very high at 550.74 milliseconds, which is well above the expected average of 100 milliseconds.
   - There is also a significant error rate of 54% encountered on the web server. This error rate is unusually high and could be a major contributing factor to the overall slow performance you are experiencing.

2. **Database Server's Performance**:
   - Upon checking the database server, it seems that there are no slow-running queries that exceed the threshold of 1000 milliseconds. This indicates that the slow performance is not due to the database queries themselves.

Given these findings, it appears the primary issue originates from the web server side of the application, which is both slow to respond and encountering a high volume of errors.

To address the immediate concerns:
- **For the web serve